In [1]:
import spacy
spacy.load('es')
from spacy.lang.es import Spanish
parser = Spanish()

In [2]:
import nltk
nltk.download('omw')
nltk.download('wordnet')

[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\Irvin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Irvin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# ******************* English ******************* #
#from nltk.corpus import wordnet as wn
#def get_lemma(word):
#    lemma = wn.morphy(word)
#    if lemma is None:
#        return word
#    else:
#        return lemma
    
#from nltk.stem.wordnet import WordNetLemmatizer
#def get_lemma2(word):
#    return WordNetLemmatizer().lemmatize(word)

In [5]:
for w in ['perros', 'corrían', 'alegremente']:
    #print(w, get_lemma(w), get_lemma2(w))
    print(w, get_stem(w))

perros perr
corrían corr
alegremente alegr


In [3]:
nltk.download('stopwords')
es_stop = set(nltk.corpus.stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Irvin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


__Experimento 1 : Stemming__

In [5]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        #elif token.orth_.startswith('@'):
        #    lda_tokens.append('USER')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_stem(word):    
    return stemmer.stem(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in es_stop]
    tokens = [get_stem(token) for token in tokens]
    return tokens

In [6]:
text_data = []
with open('irvin2.csv', encoding='utf8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        print(tokens)
        text_data.append(tokens)

['manteng', 'esper', 'instanci', 'superior', 'mayor', 'objet', 'experient', 'pod', 'judicial']
['ilusion', 'nuev', 'instanci', 'indeb', 'proces', 'conviert', 'deb', 'proces']
['despues', 'hab', 'inici', 'prision', 'prevent', 'acab', 'recib', 'notici', 'famili']
['@giullianaloz', 'final', 'despues', 'demor', 'posterg', 'promov', 'juzg', 'orig', 'apel', 'keikofujimori']
['@giullianaloz', 'final', 'elev', 'expedient', 'envi', 'incomplet', 'apel', 'oblig', 'devol']
['@giullianaloz', 'quier', 'inform', 'opinion', 'public', 'medi', 'comun', 'sig', 'afect', 'proces', 'apel']
['sig', 'camin', 'unid', 'moment', 'dificil', 'fortalec', 'famili']
['medi', 'circunst', 'dificil', 'lleg', 'celd', 'voc', 'gent', 'vien']
['llev', 'prision', 'esper', 'justici', 'prevalezc', 'instanci', 'superior']
['denigr', 'tod', 'mujer', 'viv', 'prision', 'sorprend']
['keik', 'fujimori', 'agradezc', 'profund', 'multipl', 'muestr', 'apoy', 'frent', 'agravi', 'recib']
['@redeskeik', 'mariell', 'balbi', 'fiscal', 'quier

['desp', 'loret', 'compart', 'imagen', 'visit', 'hermos', 'region', 'cuent']


In [7]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


__Para 5 topics__

In [15]:
import gensim
NUM_TOPICS = 3
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')

In [16]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.023*"amig" + 0.012*"compart" + 0.009*"usted" + 0.009*"nombr"')
(1, '0.028*"candidat" + 0.023*"keik" + 0.021*"fujimori" + 0.012*"apoy"')
(2, '0.022*"@redeskeik" + 0.019*"graci" + 0.013*"@giullianaloz" + 0.009*"lleg"')


In [17]:
new_doc = '@Jessy: @Ferdinand da la vida por el presidente fujimori'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(70, 1), (235, 1)]
[(0, 0.4318231), (1, 0.45623973), (2, 0.111937195)]


__pyLDAvis__

In [18]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')

In [19]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

__Experimento 2: Lemmatization__

In [20]:
def tokenize_and_lemmatize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        #elif token.orth_.startswith('@'):
        #    lda_tokens.append('USER')
        else:
            lda_tokens.append(token.lemma_)
    return lda_tokens

def prepare_text_for_lda(text):
    tokens = tokenize_and_lemmatize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in es_stop]
    return tokens

In [21]:
text_data = []
with open('irvin2.csv', encoding='utf8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        print(tokens)
        text_data.append(tokens)

['Mantengo', 'esperanzar', 'instancia', 'superior', 'mayor', 'objetividad', 'experiencia', 'Poder', 'Judicial']
['ilusión', 'nuevo', 'instancia', 'INDEBIDO', 'PROCESO', 'convertir', 'DEBIDO', 'PROCESO', 'decir']
['después', 'haberse', 'iniciar', 'prisión', 'preventivo', 'acabar', 'recibir', 'noticiar', 'familia']
['@GiullianaLoza', 'Finalmente', 'después', 'demorar', 'postergación', 'promover', 'juzgar', 'orígen', 'apelación', 'KeikoFujimori']
['@GiullianaLoza', 'Cuando', 'finalmente', 'elevar', 'expedientar', 'enviar', 'incompleto', 'Apelaciones', 'obligar', 'devol']
['@GiullianaLoza', 'Quiero', 'informar', 'opinión', 'público', 'medio', 'comunicación', 'seguir', 'afectar', 'procesar', 'apelación']
['Sigamos', 'caminar', 'unidad', 'momento', 'difícil', 'fortalecer', 'familia']
['mediar', 'circunstancia', 'difícil', 'llegar', 'celda', 'gente', 'comer', 'venir']
['Llevo', 'prisión', 'esperar', 'justicia', 'prevalecer', 'instancia', 'superior']
['denigrar', 'hacer', 'mujer', 'vivir', 'pr

['conocer', 'PeriodismoSelectivoIDL', 'evidenciar', 'publicar', 'audio', 'conveniencia', 'aquel']
['Comparto', 'vosotros', 'jornada', 'debatir', 'aprendizaje', 'democracia', 'interno']
['Queridos', 'amigo', 'amigo']
['celebrar', 'independencia', 'celebrar', 'vivir', 'democracia', 'vivir']
['página']
['desear', 'mejor', 'éxito', 'gestión', 'Fuerza', 'Popular', 'seguir', 'aportar', 'desarrollar', 'nacional', 'defendien']
['Felicito', 'Daniel', 'Salaverry', 'Leyla', 'Chihuán', 'Segundo', 'Tapia', 'Vilcatoma', 'integrante', 'Directiva']
['Unamos', 'esfuerzo', 'sumar', 'voluntad', 'parir', 'ReformaJudicialYa', 'VIDEO']
['ParaEvitarSuspicacias', 'Sobre', 'audio', 'Panorama', 'hablar', 'Señora', 'Fuerza']
['formar', 'joven', 'hombre', 'mujer', 'transmitir', 'conocimiento', 'valorar']
['Gracias', 'regalarnos', 'Mostraron', 'jugar', 'mundial', 'Gracias', 'profe', 'Gareca']
['CómoNoTeVoyAQuerer']
['GraciasMuchachos', 'Tenemos', 'equipar', 'selección', 'garrar']
['saludar', 'especial', 'hombre', 

In [22]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

__Para 5 topics__

In [23]:
import gensim
NUM_TOPICS = 3
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

...

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Irvin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

In [24]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.028*"haber" + 0.017*"Keiko" + 0.015*"Fujimori" + 0.008*"procesar"')
(1, '0.017*"candidato" + 0.012*"parir" + 0.011*"prisión" + 0.010*"@RedesKeiko"')
(2, '0.020*"@GiullianaLoza" + 0.008*"Fuerza" + 0.008*"@RedesKeiko" + 0.008*"SeguimosViajando"')


In [25]:
new_doc = '@Jessy: @Ferdinand da la vida por el presidente fujimori'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(272, 1)]
[(0, 0.16835204), (1, 0.663261), (2, 0.16838695)]


__pyLDAvis__

In [26]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')

In [27]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)